In [ ]:
# !git clone https://github.com/mibijoy007/EasyOCR.git

!git clone --branch train-success https://github.com/mibijoy007/EasyOCR.git



Cloning into 'EasyOCR'...
remote: Enumerating objects: 2832, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 2832 (delta 10), reused 84 (delta 5), pack-reused 2732 (from 1)
Receiving objects: 100% (2832/2832), 158.92 MiB | 15.58 MiB/s, done.
Resolving deltas: 100% (1672/1672), done.
Updating files: 100% (377/377), done.


In [ ]:
import os
print("Current Directory:", os.getcwd())

print("Files in Current Directory:", os.listdir())
#change dir
os.chdir('/content/EasyOCR/trainer')
print("New Directory:", os.getcwd())
print("Files in Current Directory:", os.listdir())


Current Directory: /content
Files in Current Directory: ['.config', 'EasyOCR', 'sample_data']
New Directory: /content/EasyOCR/trainer
Files in Current Directory: ['__pycache__', 'config_files', 'modules', 'utils.py', 'train.py', 'saved_models', 'model.py', 'py38_env', 'trainer.ipynb', 'craft', 'README.md', 'all_data', 'my_test.py', 'Untitled.ipynb', 'dataset.py']


In [ ]:
%pip install torch==1.12.0+cu113 torchvision==0.13.0+cu113 torchaudio==0.12.0 -f https://download.pytorch.org/whl/torch_stable.html


Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [ ]:
import torch

print(torch.__version__)

1.12.0+cu113


In [ ]:
print("Current Directory:", os.getcwd())

Current Directory: /content/EasyOCR/trainer


In [ ]:
# !pip install -r requirements.txt
%pip install -r ../requirements.txt

## Gpu test

In [ ]:
import torch

if torch.cuda.is_available():
    print("GPU is available:", torch.cuda.get_device_name(0))
else:
    print("GPU not available")


GPU is available: Tesla T4


In [ ]:
import os
import torch.backends.cudnn as cudnn
import yaml
from train import train
from utils import AttrDict
import pandas as pd

In [ ]:
cudnn.benchmark = True
cudnn.deterministic = False


In [ ]:


def get_config(file_path):
    with open(file_path, 'r', encoding="utf8") as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    # print('ooooopt',opt)
    if opt.lang_char == 'None':
        characters = ''
        for data in opt['select_data'].split('-'):
            csv_path = os.path.join(opt['train_data'], data, 'labels.csv')
            df = pd.read_csv(csv_path, sep='^([^,]+),', engine='python', usecols=['filename', 'words'], keep_default_na=False)
            all_char = ''.join(df['words'])
            characters += ''.join(set(all_char))
        characters = sorted(set(characters))
        opt.character= ''.join(characters)
    else:
        opt.character = opt.number + opt.symbol + opt.lang_char
    os.makedirs(f'./saved_models/{opt.experiment_name}', exist_ok=True)
    return opt


In [ ]:
%tb
opt = get_config("config_files/en_filtered_config.yaml")
print(opt)

train(opt, amp=False)
# train(opt)
#the sys.exit() error was not a problem it's just how the  training ends